In [47]:
%run BreakSectionAnalyser.ipynb
%run SectionAnalyser.ipynb
%run Metric.ipynb

class EAdvantage:
    
    def __init__(self,scale,short,mid):
        self.scale = scale
        self.short = short
        self.mid = mid
        
    def compute_E_rate_of_ma_cross_sections(self):
        analyser = SectionAnalyser(self.scale,self.short,self.mid)
        sections_df = analyser.get_all_sections_of_all_symbols(level_day)
        
        total = e_df.shape[0]
        avg_mfe = sections_df['mfe'].sum()/total
        avg_mae = sections_df['mae'].sum()/total
        
        e_rate = avg_mfe/avg_mae
        
        return e_rate
        
        
    def compute_E_rate_of_break_sections(self):
        analyser = BreakSectionAnalyser(0,self.scale,self.short,self.mid)
        sections_df = analyser.get_all_sections_of_all_symbols(level_day)
        print(str(sections_df))
        
        total = sections_df.shape[0]
        avg_mfe = sections_df['mfe'].sum()/total
        avg_mae = sections_df['mae'].sum()/total
        
        e_rate = avg_mfe/avg_mae
        
        return e_rate
    
    
#     fixed个交易日后退出
    def compute_E_rate_of_break_sections_fixed_day_exist(self,fixed):
        analyser = BreakSectionAnalyser(0,self.scale,self.short,self.mid)
        sections_df = analyser.get_all_sections_of_all_symbols(level_day)
                
        fix_day_df = pd.DataFrame(columns=['symbol','start_date','end_date','mfe','mae','e'])
        
        total = sections_df.shape[0]
        print('got '+str(total)+' sections')
        progress = 0
        for key,row in sections_df.iterrows():
            metric = Metric(row['symbol'],level_day,self.short,self.mid)
            
            start_date = row['d_s_date']
            exist_series = metric.get_fixed_day(start_date,fixed)
            if(exist_series.shape[0]<=0):
                continue
            
            exist_date = exist_series['date'].values[0]
            
            atr_value = metric.atr(start_date)
            
            max_price_date = metric.get_max_price_date(start_date,exist_date)
            min_price_date = metric.get_min_price_date(start_date,exist_date)
                
            start_price = metric.get_cur_price(start_date)
            highest_price = max_price_date['high']
            lowest_price = min_price_date['low']

            mfe = (highest_price - start_price)/atr_value
            mae = (start_price - lowest_price)/atr_value
            e = 1.0*mfe/mae
            
            fix_day_df.loc[fix_day_df.shape[0]+1]= [row['symbol'],start_date,exist_date,mfe,mae,e]
            
            progress +=1
            if(progress%50==0):
                print(round(100.0*progress/total,2),end=' ')
            
        total = fix_day_df.shape[0]
        avg_mfe = fix_day_df['mfe'].sum()/total
        avg_mae = fix_day_df['mae'].sum()/total
        
        e_rate = avg_mfe/avg_mae
        
        return e_rate
        
        

In [55]:
def grid_search():
    e_df = pd.DataFrame(columns=['short','mid','fixed','e_rate'])
    for short in [20,30,40,5,10]:
        for mid in [20,30,40,50,60,70,80,90]:        
            if(short<=mid):
                eAdvantage = EAdvantage(3,short,mid)

                last_e_rate = 0
                for fixed in [5,7,9,12,14,16,18,20,30,40,50,60,70,80,90,100,120]:
                    e_rate = eAdvantage.compute_E_rate_of_break_sections_fixed_day_exist(fixed)
                    print(str(short)+','+str(mid)+','+str(fixed)+' '+str(e_rate))

                    e_df.loc[e_df.shape[0]+1] = [short,mid,fixed,e_rate]
                    
    return e_df



        

get_all_sections_of_all_symbols done total:142
20,20,5 0.5405405405405409
get_all_sections_of_all_symbols done total:142
20,20,7 0.7027027027027043
get_all_sections_of_all_symbols done total:142
20,20,9 0.7027027027027043
get_all_sections_of_all_symbols done total:142
20,20,12 0.7027027027027043
get_all_sections_of_all_symbols done total:142
20,20,14 0.7027027027027043
get_all_sections_of_all_symbols done total:142
20,20,16 0.7027027027027043
get_all_sections_of_all_symbols done total:142
20,20,18 0.7027027027027043
get_all_sections_of_all_symbols done total:142
20,20,20 0.9054054054054049
get_all_sections_of_all_symbols done total:142
20,20,30 1.6756756756756754
get_all_sections_of_all_symbols done total:142
20,20,40 1.9324324324324336
get_all_sections_of_all_symbols done total:142
20,20,50 1.9324324324324336
get_all_sections_of_all_symbols done total:142
20,20,60 1.9324324324324336
get_all_sections_of_all_symbols done total:142
20,20,70 1.9324324324324336
get_all_sections_of_all_symb

KeyboardInterrupt: 